# Partie 3. Classification des textes : analyse de sentiments
Dans cette troisième partie du TP, nous allons développer quelques modèles pour l’analyse de sentiments (classement de textes). Le dataset utilisé est un dataset d’analyse de sentiments dans le contexte de la finance (https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis). Nous allons utiliser pandas, scikit-learn et nltk

In [1]:
import nltk
import numpy  as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
# On tokenise nous même avec un espace blanc ou un caractère de ponctuation (\b)
token_pattern = re.compile(r'(?u)\b\w\w+\b')
tokenizer = token_pattern.findall

On télécharge le dataset (https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis?resource=download) et ajoute son path (ou autre méthode pour charger le fichier avec pandas)

## 1. Préparation de données
### Question
- Importer le dataset CSV fourni en utilisant pandas et afficher les 10 premiers échantillons.

Dans mon cas, j'utilise **Colab** je dépose donc mon fichier dans mon drive et renseigne le chemin pour y acceder. Dans votre cas si il tourne en local, renseignez juste le chemin jusqu’au fichier.

In [ ]:
# Si vous travaillez sur colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Le chemin jusqu’à "data.csv"
path_data = "/content/drive/MyDrive/PhD/Activities/Enseignements/Fabron DUT2 - Introduction IA/TP5-6/data.csv"

Maintenant on importe le csv avec pandas et affiche les 10 premiers échantillons avec la fonction *head()* de pandas.

In [2]:
# On récupère le csv
# TODO
# On affiche les 10 premières lignes
# TODO

### Question
- Encoder les sentiments en utilisant **LabelBinarizer** pour avoir un encodage OneHot.

### Explications
LabelBinarizer est une classe du module scikit-learn en Python qui est utilisée pour convertir les étiquettes de classification multi-classes en une représentation binaire.

En d'autres termes, si vous avez des données étiquetées avec plusieurs classes, LabelBinarizer convertira chaque étiquette en un vecteur binaire correspondant. Chaque position du vecteur représentera une classe possible et sera soit 1 (si l'exemple appartient à cette classe) ou 0 (si l'exemple n'appartient pas à cette classe).

Par exemple, si vous avez un ensemble de données étiquetées avec les classes "*chat*", "*chien*" et "*oiseau*", LabelBinarizer convertira chaque étiquette en un vecteur binaire de longueur 3. Si un exemple est étiqueté comme "*chien*", le vecteur correspondant serait [0, 1, 0]. Si un exemple n'appartient à aucune des classes, le vecteur serait [0, 0, 0].

La sortie de LabelBinarizer peut être utilisée pour entraîner des modèles de classification binaires tels que les classificateurs de régression logistique ou de SVM, qui ne peuvent gérer que des étiquettes binaires (1 ou 0).

Dans notre cas nous souhaitons obtenir un résultat pour les labels "*positive*", "*negative*", "*neutral*" du style:

| Sentiment   | OneHot    |
|-------------|-----------|
| Positive    | [1, 0, 0] |
| Neutral     | [0, 1, 0] |
| Negative    | [0, 0, 1] |
| Autre token | [0, 0, 0] |

In [3]:
# On crée une instance de LabelBinarizer
# TODO

# fit_transform() de LabelBinarizer pour encoder les sentiments
# TODO

On regarde ce qu'il se passe et compare avec nos sentiments en texte

In [4]:
# On remarque que negative -> 0,0, 1; positive -> 1, 0, 0; neutral -> 0, 1, 0;
# TODO print

### Question
- Diviser le dataset en entrainement et test, en utilisant train_test_split, la taille du test est 30% et random_state=0.

### Aide
**train_test_split** est une fonction du module scikit-learn en Python qui permet de diviser un ensemble de données en un ensemble d'entraînement et un ensemble de test. Cette fonction est souvent utilisée dans le cadre de l'apprentissage automatique (machine learning) pour évaluer la performance d'un modèle sur des données indépendantes de celles utilisées pour l'entraînement.

La fonction train_test_split prend en entrée les données à diviser ainsi que le pourcentage de données à réserver pour l'ensemble de test. Elle renvoie quatre ensembles de données : *X_train, X_test, y_train, y_test*.

Les paramètres les plus couramment utilisés de train_test_split sont :
- **test_size** : le pourcentage de données à réserver pour l'ensemble de test. Par exemple, si test_size est fixé à 0.2, cela signifie que 20% des données seront réservées pour le test et que 80% des données seront utilisées pour l'entraînement.
- **random_state** : un entier qui permet de fixer la graine aléatoire pour la répartition des données. Cela permet de s'assurer que la même répartition des données sera obtenue à chaque exécution du code, ce qui permet de faciliter la reproductibilité des résultats.

### Rappel:
Dans le contexte de l'apprentissage automatique supervisé, X représente les variables explicatives (ou caractéristiques) du jeu de données et y représente la variable cible (ou réponse) que nous cherchons à prédire.

In [5]:
# Stocker les labels "Sentence" dans la variable X
# TODO

# Diviser l'ensemble de données en ensembles d'entraînement et de test en utilisant la fonction train_test_split de scikit-learn. 
# L'ensemble de test contiendra 30% des données et la graine aléatoire est fixée à 0 pour assurer la reproductibilité des résultats.
# TODO

Si vous êtes curieux ou dubitatif, vérifiez les données dans X et y

In [8]:
# TODO print

Puis verifiez le ratio 70% train 30% test ainsi que la correspondance de taille X-y

In [9]:
# On check les tailles si on a bien 70/30% 
# TODO print

### Question 
- Entrainer un modèle de vectorisation TF sur le texte d’entrainement et vectoriser le.

### Aide
On utilisera **CountVectorizer()**, qui est une fonction de la bibliothèque Scikit-Learn utilisée pour transformer un corpus de documents textuels en une matrice de termes de document, qui représente la fréquence des mots dans chaque document.

Plus précisément, CountVectorizer() :

- Tokenise les documents : chaque document est divisé en mots ou "tokens".
- Construit un vocabulaire : les mots uniques de tous les documents sont collectés pour créer un vocabulaire.
- Encode chaque document : chaque document est représenté par un vecteur qui compte la fréquence de chaque mot du vocabulaire dans ce document.
Le résultat est une matrice creuse (sparse matrix) où chaque ligne correspond à un document et chaque colonne correspond à un mot du vocabulaire. Les valeurs dans la matrice représentent le nombre d'occurrences de chaque mot dans chaque document.

Voici un exemple avec les 3 documents suivants:
- Le chien rouge joue avec le chat rouge
- Le chien est bleu, le chat est rouge, le chien est rouge
- Le chat est rouge

| Le         | chien | rouge | joue | avec | le | chat | est | bleu |
|------------|-------|-------|------|------|----|------|-----|------|
| Document 1 | 1     | 1     | 2    | 1    | 1  | 1    | 1   | 0    |
| Document 2 | 1     | 2     | 2    | 0    | 0  | 1    | 1   | 2    |
| Document 3 | 1     | 0     | 1    | 0    | 0  | 0    | 1   | 1    |

Cette matrice peut ensuite être utilisée comme entrée pour des algorithmes de machine learning tels que la régression logistique, les SVM, etc.

In [10]:
# Création d'une instance de CountVectorizer
# TODO

# Utilisation de fit_transform() pour vectoriser le texte d'entraînement
# TODO

Regardons un exemple de plus pres avec le premier document: "The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model ."

In [11]:
# On prends un exemple
# TODO
# On affiche la sparse matrice de notre phrase de test
# TODO print

Par exemple, le tuple (0, 1013) 2 signifie que le mot ayant l'indice 1013 dans le vocabulaire du texte d'entraînement (c'est-à-dire le mot correspondant à la 1014ème colonne de la matrice) apparaît deux fois dans la première phrase du texte d'entraînement.

Pour afficher les indices de chaque mot dans le vocabulaire correspondant aux colonnes de la matrice, vous pouvez utiliser l'attribut vocabulary_ de l'objet vectorizer, comme ceci :

In [13]:
# Afficher le dictionnaire
# X_voc = # TODO votre vectorizer.vocabulary_
# print(X_voc)

# # Si vous chercher une valeur en particulier dans le vocabulaire
# valeur_recherchee = 1013
# for cle, valeur in X_voc.items():
#     if valeur == valeur_recherchee:
#         print("L'index:", valeur_recherchee, "correspond au mot:", cle)

- En utilisant le même modèle, vectoriser le dataset de test.

### Aide
La méthode fit_transform() de CountVectorizer est une combinaison des méthodes fit() et transform(). Cette méthode est utilisée pour apprendre le vocabulaire du texte d'entraînement et vectoriser le texte en une matrice creuse (sparse matrice). Elle est utilisée pour la première étape de la création d'un modèle de classification de texte, où le vocabulaire est appris à partir du texte d'entraînement et utilisé pour transformer le texte d'entraînement en une représentation vectorielle.

La méthode transform() de CountVectorizer, quant à elle, est utilisée pour transformer le texte de test (ou tout autre texte) en une représentation vectorielle en utilisant le vocabulaire appris pendant la phase d'entraînement. Cette méthode est utilisée pour vectoriser le texte de test de la même manière que le texte d'entraînement, en utilisant le même vocabulaire.

**En résumé**, fit_transform() est utilisée pour apprendre le vocabulaire et vectoriser le texte d'entraînement, tandis que transform() est utilisée pour vectoriser le texte de test en utilisant le même vocabulaire que celui appris pendant la phase d'entraînement. La méthode fit_transform() doit être utilisée sur le texte d'entraînement, tandis que la méthode transform() doit être utilisée sur le texte de test (ou tout autre texte à vectoriser) après avoir utilisé fit_transform() sur le texte d'entraînement.

In [14]:
# Utilisation de transform() pour vectoriser le texte de test
# TODO

### Question
- Définir une fonction tokenstem qui prend un texte et qui utilise tokenizer pour avoir une liste des tokens ensuite elle génère une liste des tokens radicalisés en utilisant nltk.stem.porter.PorterStemmer et elle ne considère pas les tokens appartenant à nltk.corpus.stopwords.words(’english’)


On télécharge les données, les stop words en anglais cette fois ci:

In [17]:
%%capture
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Maintenant on crée notre fonction **tokenstem** (tokenisation + remove stopwords + stemming)

In [18]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def tokenstem(text):
    # Tokenisation des mots dans le texte
    tokens = tokenizer(text) # On utilise notre fonction de tokenisation défini plus haut
    # Création de l'objet PorterStemmer
    # Il y a d'autres méthodes que PorterStemmer, vous pouvez comparer
    # TODO
    # Liste de stopwords en anglais
    # TODO
    # Racinisation des mots et exclusion des stopwords
    # TODO
    return stems

On regarde le résultat de note superbe fonction **tokenstem**

In [19]:
# TODO print
# print(X_train.loc[0])
# print(tokenstem(X_train.loc[0]))

### Question
-  Refaire les deux étapes précédentes, mais en limitant la taille du vecteur max_features = 3000 et en utilisant tokenstem comme tokenizer.

On utilise toujours **CountVectorizer()**, regardez comment l'adapter avec nos rpérequis.

In [20]:
# Création d'une instance de CountVectorizer avec un maximum de 3000 features et en utilisant le tokenizer personnalisé (question précédente)
# TODO

# Utilisation de fit_transform() pour vectoriser le texte d'entraînement
# TODO
# Utilisation de transform() pour vectoriser le texte de test
# TODO

## 2. Entrainement
### Question
- Créer un modèle de type linear support vector machine (SVM)https://scikit-learn.org/stable/modules/svm.html#svm
- Prédire les classes en appliquant le modèle sur les représentations d’entrainement
- Prédire les classes en appliquant le modèle sur les représentations vectorielles de test

### Rappel
**SVM**, ou Support Vector Machines, est un algorithme d'apprentissage automatique supervisé utilisé pour la classification et la régression. Il est principalement utilisé pour résoudre des problèmes de classification binaire, où l'objectif est de séparer les données en deux classes distinctes.

Le principe de base de SVM consiste à trouver un hyperplan qui sépare les données en deux classes. L'hyperplan est choisi de manière à maximiser la marge entre les deux classes, c'est-à-dire la distance entre les points les plus proches des deux classes. Les points les plus proches des deux classes sont appelés vecteurs de support, d'où le nom de "Support Vector Machines".

Le type "*linear*" de SVM est utilisé lorsque les données peuvent être séparées linéairement, c'est-à-dire lorsque les deux classes peuvent être séparées par un hyperplan. Dans ce cas, l'algorithme de SVM trouve l'hyperplan optimal qui maximise la marge entre les deux classes. L'hyperplan est défini par une fonction linéaire qui prend les caractéristiques d'entrée des données en entrée et produit une sortie qui permet de déterminer la classe à laquelle appartient chaque donnée.

On implemente un SVM avec la première méthode d'entrainement des embeddings.
Vous aurez surement besoin d'adapter les données avec la fonction inverse_transform() pour recuperer les labels initiaux.

In [21]:
from sklearn.svm import SVC

# Définir le modèle SVM
# TODO

# Entraîner le modèle
# TODO

# Prédire les classes pour les données de test
# TODO

- Afficher le rapport de classification

On utilisera **classification_report**, une fonction de la bibliothèque scikit-learn en Python qui calcule les métriques de performance pour un modèle de classification. Elle prend en entrée les vraies étiquettes de classe (appelées également labels ou ground truth) et les prédictions du modèle, puis calcule un certain nombre de métriques de performance telles que la précision, le rappel, le score F1 et le support.

Les métriques de performance calculées par classification_report sont spécifiques à chaque classe. Pour chaque classe, elle calcule les métriques de performance suivantes:

- La précision (precision) : la proportion de prédictions positives correctes par rapport à toutes les prédictions positives effectuées par le modèle. Elle mesure la capacité du modèle à identifier correctement les instances de la classe donnée.

- Le rappel (recall) : la proportion d'instances positives correctement identifiées par rapport à toutes les instances positives dans les données. Elle mesure la capacité du modèle à identifier toutes les instances positives de la classe donnée.

- Le score F1 (F1 score) : une moyenne harmonique de la précision et du rappel qui donne un score unique pour chaque classe. Il est utile lorsque la précision et le rappel ont des valeurs très différentes.

- Le support (support) : le nombre d'instances réelles de la classe donnée dans les données.

classification_report calcule également ces métriques pour l'ensemble des classes, en prenant en compte la moyenne des métriques sur toutes les classes.

In [22]:
# Évaluer la performance du modèle
# TODO

### Question
- Créer un modèle de type linear support vector machine (SVM)https://scikit-learn.org/stable/modules/svm.html#svm
- Prédire les classes en appliquant le modèle sur les représentations d’entrainement
- Prédire les classes en appliquant le modèle sur les représentations vectorielles de test

Cette fois ci on experimente avec notre foction tokenstem

In [23]:
# Définir le modèle SVM
# TODO

# Entraîner le modèle
# TODO

# Prédire les classes pour les données de test
# TODO

In [24]:
# Évaluer la performance du modèle
# TODO

# Fin
Libre a vous de continuer et d'experimenter avec d'autresfonctions et paramètres. Vous pouvez par exemple utiliser d'autres méthodes de vectorisation comme: 
TfidfVectorizer : Cette fonctionnalité est également disponible dans la bibliothèque scikit-learn. Elle utilise le schéma de pondération TF-IDF pour la vectorisation de texte, qui est une technique courante pour représenter les termes dans un document en fonction de leur importance. Elle attribue des poids plus élevés aux termes qui sont fréquents dans un document et rares dans l'ensemble du corpus.

- HashingVectorizer : Cette fonctionnalité est également disponible dans la bibliothèque scikit-learn. Elle utilise une fonction de hachage pour convertir les mots en vecteurs de nombres entiers de longueur fixe. Cette fonctionnalité est plus rapide et nécessite moins de mémoire que CountVectorizer car elle ne stocke pas de vocabulaire.

- Word2Vec : Cette fonctionnalité utilise un modèle de réseau de neurones pour apprendre des représentations vectorielles denses pour les mots. Elle est capable de capturer des similitudes sémantiques et syntaxiques entre les mots et est utile pour la résolution de tâches telles que la prédiction de mots manquants et la classification de textes.

- GloVe : Cette fonctionnalité utilise une approche matricielle pour apprendre des représentations vectorielles pour les mots. Elle est similaire à Word2Vec et est également utile pour la résolution de tâches telles que la classification de textes et la prédiction de mots manquants.

- FastText : Cette fonctionnalité est développée par Facebook et utilise une méthode de n-grammes pour apprendre des représentations vectorielles pour les mots. Elle est capable de capturer des similitudes entre les mots qui partagent des sous-chaînes et est utile pour la classification de textes et la prédiction de mots manquants.